# Preparing the Dataset

In [ ]:
pip install datasets==2.14.5


INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 10.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
  Attempting uninstall: dill
    Found existing installation: dill 0.3.8
    Uninstalling dill-0.3.8:
      Successfully uninstalled dill-0.3.8
  Attempting uninstall: multiprocess
    Found existing installation: multiprocess 0.70.16
    Uninstalling multiprocess-0.70.16:
      Successfully uninstalled multiprocess-0.70.16
  Attempting uninstall: datasets
    Found

In [ ]:
from datasets import load_dataset

# Load the 2017 IWSLT dataset, English–French
dataset = load_dataset("iwslt2017", "iwslt2017-en-fr")

print(dataset)
print("Train size:", len(dataset["train"]))
print("Validation size:", len(dataset["validation"]))
print("Test size:", len(dataset["test"]))

# Access sentences
print(dataset["train"][0]["translation"]["en"])
print(dataset["train"][0]["translation"]["fr"])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/232825 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/8597 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/890 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 232825
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 8597
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 890
    })
})
Train size: 232825
Validation size: 890
Test size: 8597
Thank you so much, Chris. And it's truly a great honor to have the opportunity to come to this stage twice; I'm extremely grateful.
Merci beaucoup, Chris. C'est vraiment un honneur de pouvoir venir sur cette scène une deuxième fois. Je suis très reconnaissant.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# saving the dataset so I don't need to redownload it every time
dataset.save_to_disk("/content/drive/MyDrive/datasets/iwslt2017_en_fr")

NameError: name 'dataset' is not defined

# Loading the Dataset

In [ ]:
from datasets import load_from_disk

Dataset = load_from_disk("/content/drive/MyDrive/datasets/iwslt2017_en_fr")
print(Dataset)


DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 232825
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 8597
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 890
    })
})


# Implementing the Tokenizer

In [ ]:
#instead of importing the Tokenizer, we will code it from scratch.

class BPETokenizer:
  def __init__(self, Dataset, Vocab_size):

    self.Dataset = Dataset
    self.Vocab_size = Vocab_size

    self.Train_en = None
    self.Train_fr = None

    self.Val_en = None
    self.Val_fr = None

    self.Test_en = None
    self.Test_fr = None

    self.WordsFrequencies = {}
    self.Tokens = {"<pad>": 0, "<bos>": 1, "<eos>": 2, "<unk>": 3, "▁":4}

  def GetDatasets(self):

    self.Train_en = [sentence['translation']['en'] for sentence in self.Dataset['train']]
    self.Train_fr = [sentence['translation']['fr'] for sentence in self.Dataset['train']]

    self.Val_en = [sentence['translation']['en'] for sentence in self.Dataset['validation']]
    self.Val_fr = [sentence['translation']['fr'] for sentence in self.Dataset['validation']]

    self.Test_en = [sentence['translation']['en'] for sentence in self.Dataset['test']]
    self.Test_fr = [sentence['translation']['fr'] for sentence in self.Dataset['test']]

  def Initialize_Tokens(self):
    self.GetDatasets()
    indexation = 5
    for sentence in self.Train_en:
      for char in sentence:
        if self.Tokens.get(char, -1) == -1:
          if char != '':
            self.Tokens[char] = indexation
            indexation += 1

    for sentence in self.Train_fr:
      for char in sentence:
        if self.Tokens.get(char, -1) == -1:
          self.Tokens[char] = indexation
          indexation += 1

  def GetWordsFrequencies(self):
    for sentence in self.Train_en:
          words = sentence.strip().split()
          for word in words:
            word = "▁" + word
            if self.WordsFrequencies.get(word, -1) == -1:
              self.WordsFrequencies[word] = 1
            else:
              self.WordsFrequencies[word] += 1






  """
  datatset is a list of sentences this function returns the tokenized form
  ["I", "love", "Banana"] => [1, 65, 6542]
  it returns the list of tokens depending on the language
  """

  def Tokenize(self, dataset, lang = "en"):
    pass

  #this function gets a list of tokens and fabricates every possible pair of tokens
  def Pair_of_Tokens(self, Tokens):
    A = []
    for key1 in Tokens.keys():
      for key2 in Tokens.keys():
        A.append([key1, key2])

    return A

  # this function corresponds to an iteration over the dataset and return the current_new_token state
  def TokensIteration(self, Current_Tokens, dataset):
    pass



In [ ]:
Tokenizer = BPETokenizer(Dataset, 16000)
Tokenizer.GetDatasets()
print("finish")
Tokenizer.Initialize_Tokens()


finish


In [ ]:
print(Tokenizer.Tokens)

{'<pad>': 0, '<bos>': 1, '<eos>': 2, '<unk>': 3, '▁': 4, 'T': 5, 'h': 6, 'a': 7, 'n': 8, 'k': 9, ' ': 10, 'y': 11, 'o': 12, 'u': 13, 's': 14, 'm': 15, 'c': 16, ',': 17, 'C': 18, 'r': 19, 'i': 20, '.': 21, 'A': 22, 'd': 23, 't': 24, "'": 25, 'l': 26, 'g': 27, 'e': 28, 'v': 29, 'p': 30, 'w': 31, ';': 32, 'I': 33, 'x': 34, 'f': 35, 'b': 36, 'P': 37, 'F': 38, 'N': 39, '!': 40, 'q': 41, '-': 42, 'S': 43, 'W': 44, 'H': 45, '5': 46, '0': 47, 'D': 48, 'j': 49, 'Y': 50, '?': 51, '4': 52, 'E': 53, '2': 54, '3': 55, '8': 56, 'L': 57, '"': 58, 'V': 59, 'G': 60, ':': 61, 'z': 62, 'B': 63, 'U': 64, 'M': 65, 'J': 66, 'O': 67, '1': 68, '9': 69, 'R': 70, 'K': 71, '6': 72, '$': 73, 'Z': 74, '7': 75, '/': 76, 'X': 77, '&': 78, '[': 79, ']': 80, 'é': 81, 'ù': 82, 'Q': 83, 'â': 84, ')': 85, '(': 86, 'ć': 87, '+': 88, 'ñ': 89, 'ï': 90, 'á': 91, '♫': 92, '%': 93, '#': 94, 'í': 95, '=': 96, '’': 97, 'ó': 98, '@': 99, 'º': 100, '˚': 101, 'ö': 102, '*': 103, '–': 104, 'É': 105, '…': 106, 'ü': 107, 'ä': 108, 'ò'

In [ ]:
Tokenizer.GetWordsFrequencies()

In [ ]:
print(Tokenizer.WordsFrequencies)

In [ ]:
print(Tokenizer.Train_en[0])
words = Tokenizer.Train_en[0].strip().split()
print(words)

Thank you so much, Chris. And it's truly a great honor to have the opportunity to come to this stage twice; I'm extremely grateful.
['Thank', 'you', 'so', 'much,', 'Chris.', 'And', "it's", 'truly', 'a', 'great', 'honor', 'to', 'have', 'the', 'opportunity', 'to', 'come', 'to', 'this', 'stage', 'twice;', "I'm", 'extremely', 'grateful.']


In [ ]:
sentence = Tokenizer.Train_en[0]
n = len(sentence)
print(n)
WordsFrequencies = {}
word_begin = 0
word_end = 0
for i in range(n):
  if sentence[i] == ' ':
    print(i)
    word_begin = word_end
    word_end = i
    word = sentence[word_begin: word_end]

    print(word)
    if WordsFrequencies.get(word, -1) == -1:
      WordsFrequencies[word] = 0
    else:
      WordsFrequencies[word]+=1

print(WordsFrequencies)

131
5
Thank
9
 you
12
 so
18
 much,
25
 Chris.
29
 And
34
 it's
40
 truly
42
 a
48
 great
54
 honor
57
 to
62
 have
66
 the
78
 opportunity
81
 to
86
 come
89
 to
94
 this
100
 stage
107
 twice;
111
 I'm
121
 extremely
{'Thank': 0, ' you': 0, ' so': 0, ' much,': 0, ' Chris.': 0, ' And': 0, " it's": 0, ' truly': 0, ' a': 0, ' great': 0, ' honor': 0, ' to': 2, ' have': 0, ' the': 0, ' opportunity': 0, ' come': 0, ' this': 0, ' stage': 0, ' twice;': 0, " I'm": 0, ' extremely': 0}
